In [1]:
from importlib import reload
import pdfquery

import incident
reload(incident)
from incident import Incident
import timing
reload(timing)

ModuleNotFoundError: No module named 'pdfquery'

In [2]:
def get_text_from_curve(ltcurve):
    # text can be within LTTextBoxHorizontal or LTTextLineHorizontal
    # the problem is that these are interleaved
    # so we select EITHER as they come
    # if we chose the Boxes first and the Lines second then merged those lists,
    # the resulting list would be out of order!
    # e.g. if the true order is B1 L1 B2 B3 L2, the approach we are using
    # gives you the right order... but choosing Boxes and Lines separately
    # gives you B1 B2 B3 L1 L2!!!!
    textual_elements = ltcurve.cssselect("LTTextBoxHorizontal, LTTextLineHorizontal")
    texts = [t.text.strip() for t in textual_elements]
    
    # remove empty lines
    cleaned_texts = [t for t in texts if t != '']
    
    # PROBLEM with this approach: in rare cases some text from this falls way
    # outside the ltcurve. Might it still be within the bounding box though?
    
    # UPDATE: try gathering all data
    # for 11/28 consider these bboxes
    #
    # 10:08am [247.08, 80.197, 275.587, 90.18]
    # hp laptop [6.0, 65.784, 737.868, 82.318]
    # bounding box [3.36, 77.7, 754.86, 98.64]
    
    return cleaned_texts

In [3]:
import csv
import json

def dump_csv(incidents):
    """
    Dumps a list of Incident objects to CSV.
    """
    with open('harvard_crime_incidents.csv', 'w') as csvfile:
        fieldnames = Incident.CSV_FIELDS
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for incident in incidents:
            writer.writerow(incident.to_dict_for_csv())

In [6]:
def incidents_of_pdf(pdf_file):
    """
    Pass this function the result of a pdfquery.PDFQuery() function.
    This will read through the pdf file and return a list of 
    Incident objects contained in there!
    
    Make sure the PDF is load()'ed before you pass it.
    """
    
    # TODO watch out for things like 11/24/17 where there were no incidents. there's a specific tag for those.
    
    # so each individual report, as well as headers, is filed inside 
    # its own <LTCurve>. The text fields are inside <LTTextLineHorizontal>s and <LTTextBoxHorizontal>s
    # inside the <LTCurve>.
    reports_plus_heads = pdf.tree.findall(".//LTCurve")
    
    # extract raw incidents
    raw_incidents = [get_text_from_curve(lt) for lt in reports_plus_heads]

    # remove headers of tables
    HEADER_ROW_TEXT = ['Reported', 'Incident Type', 'Occurred', 'Location', 'Disposition']
    incidents_without_headers = [i for i in raw_incidents if i != HEADER_ROW_TEXT]
    
    # convert incidents to proper objects
    # 9 = proper length of report; anything less is malformed
    # TODO clean up — extract error checking into its own make_incident_objects() function
    incident_objects = [incident.Incident(i) for i in incidents_without_headers if len(i) == 9]
    
    return incident_objects

In [21]:
pdf = pdfquery.PDFQuery("data/112817.pdf")
pdf.load()
all_incidents = incidents_of_pdf(pdf)


In [27]:
# try another date
pdf = pdfquery.PDFQuery("data/113017.pdf")
pdf.load()
new_incidents = incidents_of_pdf(pdf)
all_incidents += new_incidents
dump_csv(all_incidents)

In [28]:
# REMAINING TODOS:
# - have a function to programmatically download HUPD crime logs
# - have another function to run through all downloaded crime logs in the `data` folder
#   (requires us to read the file system?)
# - Extract the descriptive test along with the metadata. This is somewhat harder but still
#   very important!